E-Commerce Business Analysis (2022–2024)

Este conjunto de datos presenta un análisis exhaustivo de la rentabilidad de un negocio de comercio electrónico utilizando datos transaccionales que abarcan desde 2022 hasta 2024. El conjunto de datos contiene más de 3500 pedidos en categorías de productos clave, como electrónica, accesorios y material de oficina, y cuatro regiones geográficas principales: Norte, Este, Sur y Oeste.


In [3]:
import numpy as np
import pandas as pd

In [4]:
#leer base de datos

df = pd.read_csv("ecommerce_sales_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order Date    3500 non-null   object 
 1   Product Name  3500 non-null   object 
 2   Category      3500 non-null   object 
 3   Region        3500 non-null   object 
 4   Quantity      3500 non-null   int64  
 5   Sales         3500 non-null   int64  
 6   Profit        3500 non-null   float64
dtypes: float64(1), int64(2), object(4)
memory usage: 191.5+ KB


In [5]:
df.head() #Visualizando los primeros 5 datos por categoria

,Order Date,Product Name,Category,Region,Quantity,Sales,Profit
0,2024-12-31,Printer,Office,North,4,3640,348.93
1,2022-11-27,Mouse,Accessories,East,7,1197,106.53
2,2022-05-11,Tablet,Electronics,South,5,5865,502.73
3,2024-03-16,Mouse,Accessories,South,2,786,202.87
4,2022-09-10,Mouse,Accessories,West,1,509,103.28


#Definiendo KPIs

Margen de Beneficio Neto (Profit/Sales)*100  

Insgiht: ¿Qué categorías genera más dinero real y no solo volumen de ventas?

Ticket Promedio Sales/Número Orden

Insight: ¿Cuánto gasta un cliente en promedio cada vez que genera una orden?



#Prueba Homogeneidad

Antes de responder a nuestros cuestionamientos planteados, analicemos de manera general la manera en que se relacionan las regiones en donde se realiza el estudio e-commerce

Finalidad: Comparar si las Regiones tienen patrones de consumo homógeneos para las categorías definidas.

In [6]:
#Prueba de Ji-Cuadrada de Homogeneidad
#H0: La distribución de categorías es homógenea en todas las regiones
#H1: La distribución no es homógenea por lo menos, para una región

from scipy.stats import chi2_contingency #Librería a usar para prueba de homogeneidad

#Crear la tabla de frecuencias (Observados)
tabla_contingencia = pd.crosstab(df['Region'], df['Category'])
print("La tabla de contingencia para Región y Categoría es: \n\n", tabla_contingencia, "\n")

#Ejecutar la prueba de Ji-cuadrada
chi2, p, dof, expected = chi2_contingency(tabla_contingencia)

print(f"Estadístico Ji-cuadrada: {chi2}")
print(f"p-valor: {p}")

#Interpretación de BI
if p < 0.05:
    print("Resultado: Hay evidencia de que el consumo NO es homogéneo entre regiones.")
else:
    print("Resultado: El consumo es homogéneo; las regiones se comportan igual.")

La tabla de contingencia para Región y Categoría es: 

 Category  Accessories  Electronics  Office
Region                                    
East              338          442      81
North             348          429      81
South             370          416      97
West              345          455      98 

Estadístico Ji-cuadrada: 5.373357148469919
p-valor: 0.496893377129886
Resultado: El consumo es homogéneo; las regiones se comportan igual.


Conclusion:

No se detectaron diferencias significativas en la demanda regional. Por tanto, las regiones tienden a tener una misma distribución, lo que nos permite inferir que en las 4 regiones se obtienen las mismas frecuencias por categoría.

Margen de Beneficio Neto (Profit/Sales)*100  

Insgiht: ¿Qué categorías genera más dinero real y no solo volumen de ventas?

In [7]:
#Convertir fecha a formato datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

#Calcular Margen de Beneficio
df['Margin %'] = (df['Profit'] / df['Sales']) * 100

#Resumen por Región (KPI de ventas y margen)
resumen_region = df.groupby('Region').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Margin %': 'mean'
}).reset_index()

print(resumen_region)

  Region    Sales     Profit   Margin %
0   East  2675110  464888.46  17.507327
1  North  2488773  426314.75  17.259623
2  South  2659548  458103.27  17.228104
3   West  2844450  495358.73  17.483003


Conclusiones:

West - Lider - Mayor Ventas y Ganancias Totales

East - Eficiente -Mejor Rentabilidad por Unidad

South - Promedio - Ventas Sólidas

North - Rezago - Menor volumen de negocio en general

El negocio es bastante estable, ya que la diferencia entre el margen más alto (17.51%) y el más bajo (17.23%) es de apenas 0.28%. No hay una región "en crisis" o, en contra porte, que sea una región "top" pero sí una oportunidad clara para que el Norte crezca en volumen y el Sur mejore su eficiencia operativa.

KPI

Ticket Promedio Sales/Número Orden

Insight: ¿Cuánto gasta un cliente en promedio cada vez que genera una orden?

In [8]:
#Calculando el promedio de gasto

promedio_ventas = df["Sales"].mean()
print("Promedio de Ventas Totales: ", promedio_ventas, "\n")

#Calculamos el promedio por Región
#Usamos reset_index() para convertir la Serie en DataFrame
#Renombramos la columna de resultados
promedio_df = df.groupby('Region')['Sales'].mean().reset_index(name='Promedio de Ventas')

print(promedio_df)

Promedio de Ventas Totales:  3047.966 

  Region  Promedio de Ventas
0   East         3106.980256
1  North         2900.667832
2  South         3011.945640
3   West         3167.538976


Conclusiones:

Las ventas promedio de las 3500 ventas es de 3047.98. Agrupando los promedios por Región, podemos observar que el Rango de estos es de 266.87, es decir, la diferencia entre el promedio mayor y el promedio menor.

Con esto podemos observar que los promedios no están tan alejados entre las Regiones, sin embargo conviene destacar al Norte como un área el cuál habría que impulsar sus ventas, con un promedio debajo de los 3000.

En general, no vemos discrepancias entre las Regiones, esto lo podemos confirmar en el análisis descriptivo en relación a las regiones, sus promedios y margenes de beneficios neto. Aunado a que, se realizo una prueba de homógeneidad para ver que los comportamientos entre regiones son similares. Es decir, las regiones tienen la misma tendencia de venta y ganancias pero hay áreas de oportunidad en cuánto a la Región Norte.

**ANALIZANDO LAS COLUMNAS DEL DATAFRAME**

In [10]:
#Columna Categoria

categoria = df["Category"].value_counts()
print(categoria)

#Hay 3 categorías en los productos de ventas

Category
Electronics    1742
Accessories    1401
Office          357
Name: count, dtype: int64


Convendría realizar una prueba de independencia Ji-Cuadrada, para ver si las regiones dependen o no de las categorías.

In [11]:
from scipy.stats import chi2_contingency

#Creamos la tabla de frecuencias
tabla_contingencia = pd.crosstab(df['Region'], df['Category'])

print("Tabla de Contingencia (Observados):")
print(tabla_contingencia)

Tabla de Contingencia (Observados):
Category  Accessories  Electronics  Office
Region                                    
East              338          442      81
North             348          429      81
South             370          416      97
West              345          455      98


In [12]:
#Aplicando la prueba Ji-Cuadrada de Independencia
chi2, p, dof, esperados = chi2_contingency(tabla_contingencia)

print(f"Estadístico Chi-cuadrado: {chi2:.4f}")
print(f"P-valor: {p:.4e}")
print(f"Grados de libertad: {dof}")

Estadístico Chi-cuadrado: 5.3734
P-valor: 4.9689e-01
Grados de libertad: 6


De acuerdo con el problema, nuestras hipótesis quedarían de la siguiente manera:

H0: La venta en las regiones son independientes de las categorías de los productos e-commerce

H1: La venta en las regiones son dependientes de las categorías de los productos e-commerce

Dado a que nuestro p valor es aproximadamente 0.496 y dicho valor es mayor a un nivel de significancia alpha de 0.05, hay evidencia estadística suficiente para no rechazar la hipótesis nula, es decir, las regiones son independientes de las categorías de los productos e-commerce analizados. O en otras pálabras:

No podemos afirmar que en el Norte prefieren más la electrónica que en el Sur basándote solo en estos datos, ya que la distribución es estadísticamente similar en todas partes.